import numpy as np
import pandas as pd
data = pd.read_csv('E:/BaiduNetdiskDownload/论文/论文/data.csv')
idlist = np.unique(data['id'])
data['feature'] = data['feature'].astype('str')
columns = np.unique(data['feature'])
columns = ['id', 'gender', 'age'] + columns.tolist() + ['result']
depre = pd.DataFrame(columns = columns)
depre['id'] = idlist
columns = np.unique(data['feature'])
for i in idlist:
    depre.loc[depre['id']== i, ['gender']] = data.loc[data['id'] == i]['gender'].values[0]
    depre.loc[depre['id']== i, ['age']] = data.loc[data['id'] == i]['age'].values[0]
    depre.loc[depre['id']== i, ['result']] = data.loc[data['id'] == i]['result'].values[0]
    for j in columns:
        if j in data.loc[data['id'] == i]['feature'].values:
            depre.loc[depre['id'] == i, [j]] = data[(data['id'] == i) & (data['feature'] == j)]['value'].values[0]
depre['age'] = depre['age'].apply(lambda x : x.replace('岁', ''))
depre.to_csv('E:/BaiduNetdiskDownload/论文/论文/data1.csv', index = 0)

In [1]:
import itertools
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.feature_selection import mutual_info_classif as MIF
import random
import os
# chain()可以把一组迭代对象串联起来，形成一个更大的迭代器
from itertools import chain
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers, layers, losses, metrics
from tensorflow.keras.initializers import glorot_normal, he_normal
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Embedding, Lambda, multiply, Flatten, Concatenate
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest

from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2

from deepctr.inputs import get_dense_input, create_embedding_matrix, embedding_lookup, get_dense_input, varlen_embedding_lookup, \
    get_varlen_pooling_list, mergeDict
from deepctr.layers.sequence import SequencePoolingLayer, Transformer, AttentionSequencePoolingLayer

from deepctr.feature_column import  SparseFeat, DenseFeat, VarLenSparseFeat, get_feature_names, build_input_features, get_linear_logit, DEFAULT_GROUP_NAME, input_from_feature_columns
from deepctr.layers.core import PredictionLayer, DNN
from deepctr.layers.interaction import FM, FEFMLayer, BiInteractionPooling, AFMLayer, CIN, InteractingLayer, FwFMLayer, InnerProductLayer, OutterProductLayer, FGCNNLayer, CrossNet,  CrossNetMix
from deepctr.layers.utils import concat_func, add_func, Hash, NoMask, combined_dnn_input, reduce_sum, softmax
from tensorflow.keras.layers import Layer
from tensorflow.keras.initializers import Zeros, glorot_normal, RandomNormal
import matplotlib.pyplot as plt
import matplotlib
# matplotlib.rcParams['font.family'] = 'Arial'
# matplotlib.rcParams['font.size'] = 12
# np.random.seed(1024) # seed是一个固定的整数即可
# random.seed(1024)
# os.environ['PYTHONHASHSEED'] = str(1024)

DeepCTR version 0.9.1 detected. Your version is 0.9.0.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.9.1


# 预处理

In [2]:
depre = pd.read_csv('E:/BaiduDownload/论文/论文/data2.csv')
# pd.set_option('display.max_rows', None)
depre = depre.drop(['id', 'HFR', 'MFR', 'T-G', 'RET100', 'RETwell', 'IRF', 'TIBC', 'RET-He', 'FE','GSP', 'UIBC', 'LFR'], axis = 1)
np.unique(depre['result'])
depre=depre[~depre['result'].isin(['产后抑郁','适应障碍'])]
encoder = OrdinalEncoder()
depre.loc[:, ['gender', 'result']] = encoder.fit_transform(depre[['gender', 'result']])
print(encoder.categories_)
for i in ['age', 'gender', 'result']:
    depre.loc[:, i] = depre.loc[:, i].astype('int32')
depre

[array(['女', '男'], dtype=object), array(['抑郁发作', '抑郁状态', '抑郁症'], dtype=object)]


,gender,age,A/B,A/G,ADA,AFU,AG,ALB,ALP,ALT,...,TP,TSH,TT,Tg-Ab,UA,WBC,eGFR,hs-CTNI,na,result
0,0,76,0.82,1.51,11.59,26.39,9.40,40.94,70.81,18.71,...,68.00,2.31,17.2,0.8999,340.62,5.62,84.55,0.001,141.00,2
1,0,44,1.83,1.42,22.68,21.53,9.30,41.57,59.45,23.15,...,70.78,2.80,NaN,1.2300,213.65,4.66,111.94,NaN,141.00,1
2,0,79,2.81,1.43,11.88,31.29,9.80,45.43,98.66,26.83,...,77.20,1.42,17.5,0.8999,304.05,5.60,NaN,0.006,143.00,2
3,0,79,1.24,1.22,10.44,25.16,5.70,38.43,68.37,11.35,...,69.84,3.39,17.9,0.8999,365.19,5.72,NaN,NaN,141.00,0
4,0,63,1.00,1.54,7.73,24.53,8.40,39.45,91.96,13.46,...,65.05,1.32,17.3,0.8999,247.12,3.75,94.13,NaN,140.00,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,0,19,1.95,2.08,11.14,24.80,8.90,40.55,59.57,8.43,...,60.08,1.78,17.0,3.4000,404.22,4.98,109.20,NaN,139.00,1
635,0,73,0.95,1.06,16.72,28.24,12.60,35.30,89.68,14.73,...,68.67,1.28,19.9,0.8999,228.84,6.90,81.24,0.001,140.00,2
636,0,52,1.34,1.58,6.03,19.00,9.18,39.70,35.71,7.78,...,64.77,NaN,16.7,NaN,185.39,6.14,102.73,NaN,140.37,2
637,0,13,1.26,1.63,9.21,35.65,12.98,46.79,106.79,43.82,...,75.42,2.75,17.6,0.8999,508.45,9.21,144.95,NaN,136.79,1


In [3]:
depre = depre.reset_index(drop = True)

In [4]:
for i in range(depre['result'].size):
    if depre.loc[i, 'result'] != None:
        if depre.loc[i, 'result'] == 0 or depre.loc[i, 'result'] == 1:
            depre.loc[i, 'result'] = 0
        else:
            depre.loc[i, 'result'] = 1

In [5]:
dict(depre.loc[:, 'result'].value_counts())

{0: 501, 1: 135}

In [12]:
depre1 = depre.copy()
# impute = SimpleImputer(strategy = "mean" )
# depre1.loc[:] = impute.fit_transform(depre1.loc[:])
depre1 = depre1.fillna(0)
depre1 = depre1.sample(frac=1.0,random_state=42)
depre1 = depre1.reset_index(drop = True)
data1 = depre1.drop(columns=['result'])
labe = depre1['result']
data2 = data1.copy()

In [13]:
# dat, test, lab, label1 = train_test_split(data1, labe, test_size = 0.3)
train, test, label, label1 = train_test_split(data1, labe, test_size = 0.3, random_state = 42)

In [14]:
data = train.append(test)
sparse_features=['gender', 'age']
dense_features = list(set(data.columns.tolist()).difference(set(sparse_features)))

# 离散型数据进行标签化处理
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
# 连续型数据进行归一化处理，使其范围在（0，1）
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=4)
                        for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                        for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 3.generate input deepfm_data for model
# train, test = train_test_split(deepfm_data, test_size=0.2)

deepfm_train = data.head(train.shape[0])
deepfm_test = data.tail(test.shape[0])

deepfm_train = {name:deepfm_train[name] for name in feature_names}
deepfm_test = {name:deepfm_test[name] for name in feature_names}

In [15]:
def class_balanced_sigmoid_cross_entropy(label, logits):
    y = tf.cast(label, tf.float32)
    logits = tf.cast(logits, tf.float32)
    count_neg = tf.reduce_sum(1. - y) # the number of 0 in y
    count_pos = tf.reduce_sum(y) # the number of 1 in y (less than count_neg)
    beta = count_neg / (count_neg + count_pos)

    pos_weight = beta / (1 - beta)
    cost = tf.nn.weighted_cross_entropy_with_logits(y, logits, pos_weight)
    cost = tf.reduce_mean(cost * (1 - beta))
    return cost

In [16]:
def focal_loss(gamma=2., alpha=0.25):
 
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        loss = -tf.reduce_sum(alpha * tf.pow(1. - pt_1, gamma) * tf.math.log(1e-07+pt_1)) \
           -tf.reduce_sum((1-alpha) * tf.pow( pt_0, gamma) * tf.math.log(1. - pt_0 + 1e-07))
        return loss
    return focal_loss_fixed

In [42]:
def ENFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 128, 64),
            cin_layer_size=(128, 128,), cin_split_half=True, cin_activation='relu', l2_reg_linear=0.00001,
            l2_reg_embedding=0.00001, l2_reg_dnn=0, l2_reg_cin=0, seed=1024, bi_dropout=0, dnn_dropout=0,
            dnn_activation='relu', dnn_use_bn=False, task='binary'):


    features = build_input_features(
        linear_feature_columns + dnn_feature_columns)

    inputs_list = list(features.values())

    linear_logit = get_linear_logit(features, linear_feature_columns, seed=seed, prefix='linear',
                                    l2_reg=l2_reg_linear)

    sparse_embedding_list, dense_value_list = input_from_feature_columns(features, dnn_feature_columns,
                                                                         l2_reg_embedding, seed)

    fm_input = concat_func(sparse_embedding_list, axis=1)

#     dnn_input = combined_dnn_input(sparse_embedding_list, dense_value_list)
#     dnn_output = DNN(dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout, dnn_use_bn, seed=seed)(dnn_input)
#     dnn_logit = tf.keras.layers.Dense(
#         1, use_bias=False, kernel_initializer=tf.keras.initializers.glorot_normal(seed))(dnn_output)

    bi_out = BiInteractionPooling()(fm_input)
    if bi_dropout:
        bi_out = tf.keras.layers.Dropout(bi_dropout)(bi_out, training=None)
    dnn_input = combined_dnn_input([bi_out], dense_value_list)
    dnn_output = DNN(dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout, False, seed=seed)(dnn_input)
    dnn_logit = tf.keras.layers.Dense(
        1, use_bias=False, kernel_initializer=tf.keras.initializers.glorot_normal(seed))(dnn_output)

    final_logit = add_func([linear_logit, dnn_logit])

    if len(cin_layer_size) > 0:
        exFM_out = CIN(cin_layer_size, cin_activation,
                       cin_split_half, l2_reg_cin, seed)(fm_input)
        exFM_logit = tf.keras.layers.Dense(1, kernel_initializer=tf.keras.initializers.glorot_normal(seed))(exFM_out)
        final_logit = add_func([final_logit, exFM_logit])

    output = PredictionLayer(task)(final_logit)

    model = tf.keras.models.Model(inputs=inputs_list, outputs=output)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), 
            loss=losses.BinaryCrossentropy(), 
            metrics=['AUC', 'binary_accuracy', 'Precision', 'Recall'])
    return model

In [51]:
metrics = []
lrs = np.arange(0, 1, 0.1)
for i in lrs:
    metric = []
    for j in range(3):
        model = ENFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 128, 64),
            cin_layer_size=(128, 128,), cin_split_half=True, cin_activation='relu', l2_reg_linear=0.3,
            l2_reg_embedding=0.5, l2_reg_dnn=0, l2_reg_cin=0, seed=1024, bi_dropout=0, dnn_dropout=0.2,
            dnn_activation='relu', dnn_use_bn=False, task='binary')
        input_train = deepfm_train
        model.fit(input_train,
                        label,
                        validation_split=0.2,
                        epochs=100,
                        batch_size=128,
                        shuffle = False,
                        verbose = 1, 
                        callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],)
        input_test = deepfm_test
        ans_mtx = model.predict(input_test, batch_size=100)
        loss, auc, acc, pre, rec = model.evaluate(input_test, label1)
        metric.append([loss, auc, acc, pre, rec])
    metric = np.array(metric)
    print(metric)
    metrics.append(metric)
print(metrics)

Epoch 1/100
3/3 [==============================] - 7s 2s/step - loss: 1.2468 - auc: 0.4932 - binary_accuracy: 0.6320 - precision: 0.1927 - recall: 0.3281 - val_loss: 1.2503 - val_auc: 0.3507 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 1s 308ms/step - loss: 1.0838 - auc: 0.4886 - binary_accuracy: 0.8202 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3500 - val_auc: 0.3282 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 1s 292ms/step - loss: 1.0720 - auc: 0.5007 - binary_accuracy: 0.8202 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3694 - val_auc: 0.3261 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/100
3/3 [==============================] - 1s 285ms/step - loss: 1.0537 - auc: 0.5089 - binary_accuracy: 0.8202 - precision: 0.0000e+00 - recall: 0.

6/6 [==============================] - 0s 79ms/step - loss: 0.7979 - auc: 0.5994 - binary_accuracy: 0.7696 - precision: 0.0000e+00 - recall: 0.0000e+00 0s - loss: 0.7823 - auc: 0.6554 - binary_accuracy: 0.7625 - precision: 0.0000e+00 - recall: 0.0000
Epoch 1/100
3/3 [==============================] - 9s 3s/step - loss: 1.2142 - auc: 0.4905 - binary_accuracy: 0.6938 - precision: 0.1429 - recall: 0.1406 - val_loss: 1.2582 - val_auc: 0.3381 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 1s 379ms/step - loss: 1.0787 - auc: 0.4856 - binary_accuracy: 0.8202 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3528 - val_auc: 0.3173 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 1s 383ms/step - loss: 1.0689 - auc: 0.4981 - binary_accuracy: 0.8202 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3480 - val_

6/6 [==============================] - 1s 97ms/step - loss: 0.7863 - auc: 0.5783 - binary_accuracy: 0.7330 - precision: 0.2143 - recall: 0.0698
Epoch 1/100
3/3 [==============================] - 8s 3s/step - loss: 1.2734 - auc: 0.4938 - binary_accuracy: 0.5843 - precision: 0.1769 - recall: 0.3594 - val_loss: 1.2803 - val_auc: 0.2986 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 2s 606ms/step - loss: 1.0805 - auc: 0.4868 - binary_accuracy: 0.8202 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.4047 - val_auc: 0.2804 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 1s 360ms/step - loss: 1.0816 - auc: 0.4893 - binary_accuracy: 0.8202 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3705 - val_auc: 0.2693 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/100


6/6 [==============================] - 1s 86ms/step - loss: 0.7820 - auc: 0.5945 - binary_accuracy: 0.7592 - precision: 0.0000e+00 - recall: 0.0000e+00
[[0.79791534 0.59938717 0.76963353 0.         0.        ]
 [0.78631949 0.57825267 0.7329843  0.21428572 0.06976745]
 [0.78203768 0.59451604 0.75916231 0.         0.        ]]
Epoch 1/100
3/3 [==============================] - 9s 3s/step - loss: 1.1694 - auc: 0.4984 - binary_accuracy: 0.7697 - precision: 0.1786 - recall: 0.0781 - val_loss: 1.3193 - val_auc: 0.3407 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 1s 368ms/step - loss: 1.0807 - auc: 0.5097 - binary_accuracy: 0.8202 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3908 - val_auc: 0.3197 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 2s 617ms/step - loss: 1.0768 - auc: 0.4573 - binary_accuracy: 0

6/6 [==============================] - 1s 91ms/step - loss: 0.7465 - auc: 0.5811 - binary_accuracy: 0.7749 - precision: 0.5000 - recall: 0.0233- ETA: 0s - loss: 0.7320 - auc: 0.6279 - binary_accuracy: 0.7688 - precision: 0.5000 - recall: 0.
Epoch 1/100
3/3 [==============================] - 9s 3s/step - loss: 1.2465 - auc: 0.4945 - binary_accuracy: 0.6348 - precision: 0.1700 - recall: 0.2656 - val_loss: 1.2494 - val_auc: 0.3627 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 1s 409ms/step - loss: 1.0837 - auc: 0.5135 - binary_accuracy: 0.8202 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3449 - val_auc: 0.3671 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 1s 408ms/step - loss: 1.0785 - auc: 0.4533 - binary_accuracy: 0.8202 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3594 - val_auc: 0.377

...

6/6 [==============================] - 1s 97ms/step - loss: 0.6834 - auc: 0.4874 - binary_accuracy: 0.7749 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 1/100
3/3 [==============================] - 7s 2s/step - loss: 2.2257 - auc: 0.4234 - binary_accuracy: 0.3567 - precision: 0.1429 - recall: 0.5156 - val_loss: 1.2696 - val_auc: 0.3457 - val_binary_accuracy: 0.6629 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 2s 562ms/step - loss: 1.6450 - auc: 0.5021 - binary_accuracy: 0.5225 - precision: 0.1845 - recall: 0.4844 - val_loss: 1.2403 - val_auc: 0.3226 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 1s 361ms/step - loss: 1.4617 - auc: 0.4762 - binary_accuracy: 0.6096 - precision: 0.1622 - recall: 0.2812 - val_loss: 1.2408 - val_auc: 0.3364 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/100
3/3 [===

6/6 [==============================] - 0s 80ms/step - loss: 1.1539 - auc: 0.4672 - binary_accuracy: 0.7801 - precision: 1.0000 - recall: 0.0233
[[0.68552715 0.48617223 0.77486908 0.         0.        ]
 [0.68342918 0.48735073 0.77486908 0.         0.        ]
 [1.15393329 0.46723759 0.7801047  1.         0.02325581]]
Epoch 1/100
3/3 [==============================] - 7s 2s/step - loss: 3.0178 - auc: 0.5220 - binary_accuracy: 0.4747 - precision: 0.1813 - recall: 0.5469 - val_loss: 1.2782 - val_auc: 0.4060 - val_binary_accuracy: 0.5955 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 1s 376ms/step - loss: 2.9930 - auc: 0.4967 - binary_accuracy: 0.5225 - precision: 0.1807 - recall: 0.4688 - val_loss: 1.2374 - val_auc: 0.3850 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 1s 381ms/step - loss: 2.7623 - auc: 0.5565 - binary_accuracy: 0.5309 - precisio

6/6 [==============================] - 0s 80ms/step - loss: 1.1366 - auc: 0.4819 - binary_accuracy: 0.7801 - precision: 1.0000 - recall: 0.0233
Epoch 1/100
3/3 [==============================] - 7s 2s/step - loss: 2.9764 - auc: 0.5521 - binary_accuracy: 0.5056 - precision: 0.2021 - recall: 0.5938 - val_loss: 1.2659 - val_auc: 0.4271 - val_binary_accuracy: 0.6517 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 1s 371ms/step - loss: 3.0163 - auc: 0.4441 - binary_accuracy: 0.4944 - precision: 0.1548 - recall: 0.4062 - val_loss: 1.2362 - val_auc: 0.4154 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 1s 386ms/step - loss: 2.6161 - auc: 0.5774 - binary_accuracy: 0.5309 - precision: 0.2057 - recall: 0.5625 - val_loss: 1.2227 - val_auc: 0.4008 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/100
3/3 [===========

6/6 [==============================] - 0s 77ms/step - loss: 1.1438 - auc: 0.4551 - binary_accuracy: 0.7801 - precision: 1.0000 - recall: 0.0233+ - ETA: 0s - loss: 1.0913 - auc: 0.4251 - binary_accuracy: 0.8542 - precision: 1.0000 - recall: 0.0667   - ETA: 0s - loss: 1.1438 - auc: 0.4551 - binary_accuracy: 0.7801 - precision: 1.0000 - recall: 0.02
Epoch 1/100
3/3 [==============================] - 8s 3s/step - loss: 2.9060 - auc: 0.5044 - binary_accuracy: 0.5309 - precision: 0.2090 - recall: 0.5781 - val_loss: 1.2755 - val_auc: 0.3659 - val_binary_accuracy: 0.6517 - val_precision: 0.2000 - val_recall: 0.0357
Epoch 2/100
3/3 [==============================] - 1s 380ms/step - loss: 2.7207 - auc: 0.4985 - binary_accuracy: 0.5365 - precision: 0.1902 - recall: 0.4844 - val_loss: 1.2399 - val_auc: 0.3528 - val_binary_accuracy: 0.6854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 1s 372ms/step - loss: 2.2308 - auc: 0.5459 - binary_accur

6/6 [==============================] - 0s 74ms/step - loss: 1.1313 - auc: 0.4668 - binary_accuracy: 0.7801 - precision: 1.0000 - recall: 0.0233  - ETA: 0s - loss: 1.1321 - auc: 0.4596 - binary_accuracy: 0.7812 - precision: 1.0000 - recall: 0.03 - ETA: 0s - loss: 1.1400 - auc: 0.4279 - binary_accuracy: 0.7750 - precision: 1.0000 - recall: 0.
[[1.13659894 0.4819296  0.7801047  1.         0.02325581]
 [1.1438489  0.4551383  0.7801047  1.         0.02325581]
 [1.13134885 0.46684474 0.7801047  1.         0.02325581]]
[array([[0.79791534, 0.59938717, 0.76963353, 0.        , 0.        ],
       [0.78631949, 0.57825267, 0.7329843 , 0.21428572, 0.06976745],
       [0.78203768, 0.59451604, 0.75916231, 0.        , 0.        ]]), array([[0.7464956 , 0.58108109, 0.77486908, 0.5       , 0.02325581],
       [0.75923735, 0.58202386, 0.76439792, 0.25      , 0.02325581],
       [0.73478353, 0.57880265, 0.76439792, 0.41666666, 0.11627907]]), array([[0.70446169, 0.58328092, 0.77486908, 0.        , 0.     

In [52]:
print(metrics)
print(*zip(lrs, np.mean(metrics, axis = 1)))

[array([[0.79791534, 0.59938717, 0.76963353, 0.        , 0.        ],
       [0.78631949, 0.57825267, 0.7329843 , 0.21428572, 0.06976745],
       [0.78203768, 0.59451604, 0.75916231, 0.        , 0.        ]]), array([[0.7464956 , 0.58108109, 0.77486908, 0.5       , 0.02325581],
       [0.75923735, 0.58202386, 0.76439792, 0.25      , 0.02325581],
       [0.73478353, 0.57880265, 0.76439792, 0.41666666, 0.11627907]]), array([[0.70446169, 0.58328092, 0.77486908, 0.        , 0.        ],
       [0.69337845, 0.6066153 , 0.77486908, 0.        , 0.        ],
       [0.69140959, 0.60087991, 0.77486908, 0.        , 0.        ]]), array([[0.67195225, 0.60025144, 0.77486908, 0.        , 0.        ],
       [0.62214535, 0.61785042, 0.77486908, 0.        , 0.        ],
       [0.61680841, 0.62696421, 0.77486908, 0.        , 0.        ]]), array([[0.62951446, 0.57306725, 0.77486908, 0.        , 0.        ],
       [0.61538589, 0.56544626, 0.77486908, 0.        , 0.        ],
       [0.66006136, 0.548